In [2]:
import pandas as pd
import time
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from tabpfn import TabPFNClassifier
import numpy as np
from matplotlib import pyplot as plt
import warnings
from skopt import BayesSearchCV, space
from sklearn.metrics import average_precision_score
# use feature importance for feature selection
from numpy import sort
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
warnings.filterwarnings("ignore")
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
### Parameters
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularPredictor
from autogluon.common import space
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
##import ts_fc_functions as ff
import import_ipynb
from darts import TimeSeries
from darts.models import NBEATSModel, XGBModel, TiDEModel, LightGBMModel, AutoARIMA
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape, r2_score, mae, rmse
from darts.utils.likelihood_models import QuantileRegression
import yaml
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import LaplaceLikelihood
import random
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error




In [4]:
def process_ts(data,start_prediction,value_to_predict,min_value,persistence_use_case,persistence_time_lag):
    if start_prediction == '0h':
        time_lag = 'time_lag_2'
    elif start_prediction == '24h':
        time_lag = 'time_lag_1'
    else:
         time_lag = 'time_lag_target'

    
    data_sorted = data.sort_values(by=['encounter_id', 'recorded_time'])
    # Identify the first occurrence where time_lag_target is True for each encounter_id
    first_true = data_sorted[data_sorted[time_lag]].groupby('encounter_id').first().reset_index()
    first_true = first_true[['encounter_id', 'recorded_time']]
    # Identify the last occurrence where time_lag_target is True for each encounter_id
    last_true = data_sorted[data_sorted['time_lag_target']].groupby('encounter_id').last().reset_index()
    last_true = last_true[['encounter_id', 'recorded_time']]
    # Merge to get the first true recorded_time back to the original DataFrame
    data_with_first_true = pd.merge(data_sorted, first_true, on='encounter_id', suffixes=('', '_first_true'))
    data_with_last_true = pd.merge(data_sorted, last_true, on='encounter_id', suffixes=('', '_last_true'))

    # Flag rows to delete
    data_with_first_true['delete_flag'] = (data_with_first_true['recorded_time'] > data_with_first_true['recorded_time_first_true']) & ~data_with_first_true[time_lag]
    data_with_last_true['delete_flag'] = (data_with_last_true['recorded_time'] > data_with_last_true['recorded_time_last_true']) & ~data_with_last_true['time_lag_target']

    # Filter out rows to delete
    data = data_with_first_true[~data_with_first_true['delete_flag']].drop(columns=['delete_flag', 'recorded_time_first_true'])
    # Ensure 'value' is not missing and 'time_lag_1' is properly boolean
    data = data.dropna(subset=[value_to_predict])  # Drop rows where 'value' is missing if necessary
    data_last = data_with_last_true[~data_with_last_true['delete_flag']].drop(columns=['delete_flag', 'recorded_time_last_true'])
    data_last = data_last.dropna(subset=[value_to_predict])  # Drop rows where 'value' is missing if necessary

    # Filter DataFrame to include only rows where time_lag_1 is True
    # Identify encounter_ids where time_lag_target is True at least once
    condition_1_encounter_ids = data[data[time_lag]].groupby('encounter_id').size().index
    condition_1_encounter_ids_last = data_last[data_last[time_lag]].groupby('encounter_id').size().index

    if persistence_use_case == True:
        time_lag_1_true_df = data[data[persistence_time_lag]]
        time_lag_1_true_df_last = data_last[data_last[persistence_time_lag]]

        # Check the maximum value where time_lag_1 is True for each group
        max_values = time_lag_1_true_df.groupby('encounter_id')[value_to_predict].max()
        # Identify encounter_ids where the maximum value is >= 38
        condition_2_encounter_ids = max_values[max_values >= min_value].index
        # Combine both conditions
        final_encounter_ids = set(condition_1_encounter_ids) & set(condition_2_encounter_ids)
        # last
        time_lag_1_last_df = data_last[data_last[persistence_time_lag]]
        # Check the maximum value where time_lag_1 is True for each group
        max_values = time_lag_1_true_df_last.groupby('encounter_id')[value_to_predict].max()
        # Identify encounter_ids where the maximum value is >= 38
        condition_2_encounter_ids = max_values[max_values >= min_value].index
        # Combine both conditions
        final_encounter_ids_full = set(condition_1_encounter_ids_last) & set(condition_2_encounter_ids)
    else:
    # Filter the data based on the final_encounter_ids
        final_encounter_ids = set(condition_1_encounter_ids)
        final_encounter_ids_full = set(condition_1_encounter_ids_last)

    filtered_data = data[data['encounter_id'].isin(final_encounter_ids)]
    filtered_data_full = data_last[data_last['encounter_id'].isin(final_encounter_ids_full)]

    return filtered_data, filtered_data_full

In [5]:
def filter_station(filtered_data,station):
    station_ = 'station_'+station
    df_stations = pd.read_csv("./data/station_groups_new.csv")
    df_stations = df_stations.drop_duplicates()
    df_stations = pd.get_dummies(df_stations, columns=["station_group"], prefix="station")
    grouped = df_stations.groupby('encounter_id')
    df_stations = grouped.max()
    df_stations.reset_index(inplace=True)
    df = filtered_data.merge(df_stations, on='encounter_id', how='left')
    df.drop(columns=['subject_reference_y'], inplace=True)
    df.rename(columns={'subject_reference_x': 'subject_reference'}, inplace=True)
    df = df[df[station_] == True]
    return df

In [6]:

def resample_ts(df,resample_rate,min_ts_length, imputation,max_train_length,value_to_predict,covariates):
    df['recorded_time'] = pd.to_datetime(df['recorded_time'])
    # Define the aggregation functions for numerical columns
    if covariates == True:
        agg_funcs = {
            'value': 'max',
            'crp': 'max',
            'heart_rate': 'max',
            'mean_arterial_pressure': 'max',
            'so2': 'max',
            'bili': 'max',
            'ggt': 'max',
            'hb': 'max',
            'krea': 'max',
            'leua': 'max',
            #'fourier_mean_magnitude': 'max',
            #'fourier_std_dev_magnitude': 'max',
            #'fourier_max_magnitude': 'max',
            #'fourier_min_magnitude': 'max'
        }
    else:
        agg_funcs = {
            'crp': 'max'
            #'fourier_mean_magnitude': 'max',
            #'fourier_std_dev_magnitude': 'max',
            #'fourier_max_magnitude': 'max',
            #'fourier_min_magnitude': 'max'
        }
    
    # Define a function to resample, aggregate, and interpolate within each group
    def resample_aggregate_and_interpolate(group):
        group = group.set_index('recorded_time')
        resampled = group.resample(resample_rate).agg(agg_funcs)
        resampled['encounter_id'] = group['encounter_id'].iloc[0]  # Add encounter_id back
        resampled = resampled.reset_index()
        
        if imputation == 'interpolate':
            # Interpolate missing values linearly within each group
            resampled = resampled.groupby('encounter_id').apply(
                lambda x: x.interpolate(method='linear', limit_direction='forward')
            ).reset_index(drop=True)
        elif imputation == 'forward_fill':
                # Forward fill missing values within each group
            resampled = resampled.groupby('encounter_id').apply(
                lambda x: x.ffill()
            ).reset_index(drop=True)
            resampled = resampled.groupby('encounter_id').apply(
                lambda x: x.bfill()
            ).reset_index(drop=True)
            resampled = resampled.fillna(0)
        else:
            pass

        return resampled
    
    # Apply the function to each group and combine results
    resampled_df = df.groupby('encounter_id').apply(resample_aggregate_and_interpolate).reset_index(drop=True)

    data_sorted = resampled_df.sort_values(by=['encounter_id', 'recorded_time'])
    
    # Group by 'encounter_id' and get the last 10 rows for each group
    data_recent = data_sorted.groupby('encounter_id').tail(max_train_length)

    # Reset index if needed
    resampled_df = data_recent.reset_index(drop=True)
    resampled_df = resampled_df[resampled_df[value_to_predict].notna()]

    #resampled_df = resampled_df.dropna(subset=['crp'])
    # Filter out encounter_ids with less than n data points
    valid_encounters = resampled_df['encounter_id'].value_counts()
    valid_encounters = valid_encounters[valid_encounters >= min_ts_length].index
    resampled_df = resampled_df[resampled_df['encounter_id'].isin(valid_encounters)]

    return resampled_df


In [7]:
def train_test(resampled_df,prediction_length):
    # Split the data into train and test sets
    def split_train_test(group):
        test_rows = group.nlargest(prediction_length, 'recorded_time')
        train_rows = group.drop(test_rows.index)
        return train_rows, test_rows
    
    train_list = []
    test_list = []
    
    # Group by encounter_id and split each group
    for name, group in resampled_df.groupby('encounter_id'):
        train_rows, test_rows = split_train_test(group)
        train_list.append(train_rows)
        test_list.append(test_rows)
    
    # Concatenate the lists into DataFrames
    train_data = pd.concat(train_list).reset_index(drop=True)
    test_data = pd.concat(test_list).reset_index(drop=True)
    return train_data, test_data
    

In [8]:

def cross_validate_model(df,prediction_length,data,value_to_predict,resample_rate,metric):
    n_splits=5
    num_trials=1
    unique_encounter_ids = df['encounter_id'].unique()
    # Perform multiple trials
    for trial in range(num_trials):
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=trial)
        # Split data for this trial
        for train_index, val_index in kf.split(unique_encounter_ids):
            train_encounter_ids = unique_encounter_ids[train_index]
            val_encounter_ids = unique_encounter_ids[val_index]
            
            # Select rows based on encounter_id
            training = df[df['encounter_id'].isin(train_encounter_ids)]
            validation = df[df['encounter_id'].isin(val_encounter_ids)]

            train_data_df, test_data_df = train_test(validation,prediction_length)
            train_data_df_, test_data_df_ = train_test(training,prediction_length)
            #return train_data_df, test_data_df,train_data_df_, test_data_df_
            #predictor, train_data, test_data = train_model(train_data_df_,test_data_df_)
            predictor, train_data, test_data = train_model(resampled_df, prediction_length, value_to_predict, resample_rate,metric)
            train_data_ = df[['encounter_id', 'recorded_time', value_to_predict, 'value','bili','leua','krea','ggt','heart_rate','mean_arterial_pressure','so2']]
            train_data_['recorded_time'] = train_data_['recorded_time'].dt.tz_localize(None)
            test_data_ = test_data_df[['encounter_id', 'recorded_time', value_to_predict, 'value','bili','leua','krea','ggt','heart_rate','mean_arterial_pressure','so2']]
            test_data_['recorded_time'] = test_data_['recorded_time'].dt.tz_localize(None)
            data['recorded_time'] = pd.to_datetime(data['recorded_time'])
            validation['recorded_time'] = validation['recorded_time'].dt.tz_localize(None)

            
            train_data = TimeSeriesDataFrame.from_data_frame(
                train_data_,
                id_column="encounter_id",
                timestamp_column="recorded_time"
            )
            test_data = TimeSeriesDataFrame.from_data_frame(
                test_data_,
                id_column="encounter_id",
                timestamp_column="recorded_time"
            )
            validation_data_ = TimeSeriesDataFrame.from_data_frame(
                validation,
                id_column="encounter_id",
                timestamp_column="recorded_time"
            )

            
            
            predictions = predictor.predict(train_data,model='RecursiveTabular')
            return train_data, test_data, predictions, predictor, train_data_df,  test_data_df,validation_data_,training,validation,train_data_df_,test_data_df_
            
            

            

In [1]:
def train_model(resampled_df, prediction_length, value_to_predict, resample_rate,metric):
    train_data_df, test_data_df = train_test(resampled_df,prediction_length)
    #train_data_ = train_data_df[['encounter_id', 'recorded_time', value_to_predict, 'value','bili','leua','krea','ggt','heart_rate','mean_arterial_pressure','so2']]
    train_data_ = train_data_df[['encounter_id', 'recorded_time', value_to_predict]]

    train_data_['recorded_time'] = train_data_['recorded_time'].dt.tz_localize(None)
    #test_data_ = resampled_df[['encounter_id', 'recorded_time', value_to_predict, 'value','bili','leua','krea','ggt','heart_rate','mean_arterial_pressure','so2']]
    test_data_ = resampled_df[['encounter_id', 'recorded_time', value_to_predict]]

    test_data_['recorded_time'] = resampled_df['recorded_time'].dt.tz_localize(None)
    train_data = TimeSeriesDataFrame.from_data_frame(
        train_data_,
        id_column="encounter_id",
        timestamp_column="recorded_time"
    )
    test_data = TimeSeriesDataFrame.from_data_frame(
        test_data_,
        id_column="encounter_id",
        timestamp_column="recorded_time"
    )
    predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="autogluon",
    target=value_to_predict,
    eval_metric=metric,
    freq=resample_rate,
    verbosity=3,
    quantile_levels = [0.05,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95],
    #known_covariates_names=['value','bili','leua','krea','ggt','heart_rate','mean_arterial_pressure','so2']

)

    
    predictor.fit(
        train_data,
        presets="best_quality",
        #presets="chronos_ensemble",
        time_limit=600,
        hyperparameters={
        #"DirectTabularModel": {"tabular_hyperparameters": {"XGB":{} }},
        #"RecursiveTabularModel": {"tabular_hyperparameters": {"XGB":{} }},
        #"AverageModel":{},
        #"AutoARIMA":{},
        "TiDE":{},
        "DeepAR": {},
        "PatchTSTModel": {},
        #"WaveNetModel": {},
        "TemporalFusionTransformerModel": {},
        "DLinearModel":{},
        #            "hidden_size": space.Int(20, 100),
        #            "dropout_rate": space.Categorical(0.1, 0.3)
        #},     
        #NBeatsCustomModel: {},    
        #"DeepAR":{},
        #"Chronos": {
        #    "model_path": "small",
        #    "batch_size": 64,
        #"optimization_strategy": "openvino",
        #},
        #"Chronos": {            
        #        "model_path": "tiny",
        #        "batch_size": 64,
        #        "device": "cpu"},

       },
    hyperparameter_tune_kwargs=None,
    enable_ensemble=True,
    )
    return predictor, train_data, test_data


In [7]:
def ts_dictionary(resampled_df, prediction_length):
    def remove_timezone_and_set_index(df):
        df['recorded_time'] = df['recorded_time'].dt.tz_localize(None)
        return df.set_index('recorded_time')

    def create_timeseries_dict(grouped_data):
        series_dict = {}
        for encounter_id, group in grouped_data:
            group = group.sort_values(by='recorded_time')
            ts_series = TimeSeries.from_dataframe(group, value_cols='crp', freq='D')
            series_dict[encounter_id] = ts_series
        return series_dict

    train_data_df, test_data_df = train_test(resampled_df, prediction_length)
    train_data_df = remove_timezone_and_set_index(train_data_df)
    test_data_df = remove_timezone_and_set_index(test_data_df)

    resampled_df = resampled_df.reset_index()
    resampled_df = remove_timezone_and_set_index(resampled_df)

    grouped_train = train_data_df.groupby('encounter_id')
    grouped_test = test_data_df.groupby('encounter_id')
    grouped_validation = resampled_df.groupby('encounter_id')

    train_series_dict = create_timeseries_dict(grouped_train)
    test_series_dict = create_timeseries_dict(grouped_test)
    valid_series_dict = create_timeseries_dict(grouped_validation)

    return valid_series_dict, test_series_dict, train_series_dict

In [ ]:
def ts_dictionary_covariates(resampled_df, prediction_length,covariates_list):
    def remove_timezone_and_set_index(df):
        df['recorded_time'] = df['recorded_time'].dt.tz_localize(None)
        return df.set_index('recorded_time')

    def create_timeseries_dict(grouped_data):
        series_dict = {}
        for encounter_id, group in grouped_data:
            group = group.sort_values(by='recorded_time')
            ts_series = TimeSeries.from_dataframe(group, value_cols=covariates_list, freq='D')
            series_dict[encounter_id] = ts_series
        return series_dict

    train_data_df, test_data_df = train_test(resampled_df, prediction_length)
    train_data_df = remove_timezone_and_set_index(train_data_df)
    test_data_df = remove_timezone_and_set_index(test_data_df)

    resampled_df = resampled_df.reset_index()
    resampled_df = remove_timezone_and_set_index(resampled_df)

    grouped_train = train_data_df.groupby('encounter_id')
    grouped_test = test_data_df.groupby('encounter_id')
    grouped_validation = resampled_df.groupby('encounter_id')

    train_series_dict = create_timeseries_dict(grouped_train)
    test_series_dict = create_timeseries_dict(grouped_test)
    valid_series_dict = create_timeseries_dict(grouped_validation)

    return valid_series_dict, test_series_dict, train_series_dict

In [5]:

def split_train_validation(train_series_dict, split_ratio=0.8):
    """
    Splits each TimeSeries in train_series_dict into training and validation series.
    
    Args:
        train_series_dict (dict): A dictionary with keys as identifiers and values as TimeSeries objects.
        split_ratio (float): The ratio of data to use for training. The rest will be used for validation.

    Returns:
        dict, dict: Two dictionaries containing the training and validation series.
    """
    train_split_dict = {}
    val_split_dict = {}

    for key, series in train_series_dict.items():
        # Calculate the split point
        split_point = int(len(series) * split_ratio)
        
        # Split the series
        train_series, val_series = series.split_before(series.time_index[split_point])
        
        # Store the results
        train_split_dict[key] = train_series
        val_split_dict[key] = val_series
    
    return train_split_dict, val_split_dict

In [4]:


def pad_time_series(ts, max_length, pad_value=0):
    """
    Pads a TimeSeries object along its time dimension to max_length with the pad_value,
    extending the time series into the past.

    Args:
        ts (TimeSeries): The TimeSeries object to pad.
        max_length (int): The length to pad the time series to.
        pad_value (numeric): The value used for padding.

    Returns:
        TimeSeries: A new TimeSeries object with padded data.
    """
    # Convert TimeSeries to pandas DataFrame
    df = ts.pd_dataframe()
    
    # Extract current length and time index
    current_length = df.shape[0]
    time_index = df.index
    
    if current_length >= max_length:
        raise ValueError(f"Current length of time series is already >= max_length.")
    
    # Compute padding
    padding_length = max_length - current_length
    
    # Create new time index for padded data
    start_time = time_index[0]
    new_time_index = pd.date_range(start=start_time - pd.Timedelta(days=padding_length), periods=padding_length, freq='D')
    
    # Pad the data along the time dimension (prepend padding)
    padded_data = np.pad(df.values, ((padding_length, 0), (0, 0)), 'constant', constant_values=pad_value)
    
    # Create new DataFrame with padded data
    padded_df = pd.DataFrame(padded_data, columns=df.columns, index=new_time_index.append(time_index))
    
    # Convert the padded DataFrame back to TimeSeries
    padded_ts = TimeSeries.from_dataframe(padded_df)
    
    return padded_ts

def pad_series_dict(series_dict, max_length, pad_value=0):
    """
    Pads each TimeSeries in the dictionary to the max_length with the pad_value.

    Args:
        series_dict (dict): A dictionary where the keys are series identifiers
                            and the values are TimeSeries objects.
        max_length (int): The length to pad each series to.
        pad_value (numeric): The value used for padding.

    Returns:
        dict: A dictionary of padded TimeSeries.
    """
    padded_series_dict = {}
    for key, ts in series_dict.items():
        padded_series = pad_time_series(ts, max_length, pad_value)
        padded_series_dict[key] = padded_series
    return padded_series_dict





# Now padded_series_dict contains the padded TimeSeries objects


In [1]:
def fit_model_vanilla(model,train_series_dict,valid_series_dict_full,covariates,train_series_covars):
    if covariates == True:
        model.fit(series=list(train_series_dict.values()),past_covariates=list(train_series_covars.values()),val_series=list(valid_series_dict_full.values()))
        
    else:
        model.fit(series=list(train_series_dict.values()))
    return model

In [ ]:
def fit_stats_model_vanilla(model,train_series_dict,valid_series_dict_full,covariates,train_series_covars):
    if covariates == True:
        model.fit(series=list(train_series_dict.values()),past_covariates=list(train_series_covars.values()), val_series=list(valid_series_dict_full.values()))
        
    else:
        model.fit(series=list(train_series_dict.values()),val_series=list(valid_series_dict_full.values()))
    return model

In [1]:
def model_configs(min_train_length,prediction_length,max_train_length,padding):
    if padding == True:
        training = max_train_length-prediction_length
    else:
        training = min_train_length-prediction_length

    #
    model_nbeats = NBEATSModel(
    input_chunk_length=training,
    output_chunk_length=prediction_length,
    generic_architecture=True,
    num_stacks=10,
    num_blocks=1,
    num_layers=4,
    layer_widths=512,
    n_epochs=500,
    nr_epochs_val_period=1,
    batch_size=800,
    likelihood=LaplaceLikelihood()
    )
    #
    model_xgb = XGBModel(
    lags=training,
    output_chunk_length=prediction_length, 
    force_col_wise='true',
    use_static_covariates=False,
    multi_models=True)
    #
    model_lgb = LightGBMModel(
    lags=training,
    output_chunk_length=prediction_length, 
    force_col_wise='true',
    use_static_covariates=False,
    likelihood="quantile",
    quantiles=[0.05, 0.1, 0.5, 0.9, 0.95],
    #quantiles=[0.49, 0.5, 0.51],

    multi_models=False)
    #
    model_tide = TiDEModel(
    input_chunk_length=training,
    output_chunk_length=prediction_length
    )
    model_aarima = AutoARIMA()
    return model_nbeats, model_xgb, model_lgb, model_tide, model_aarima


In [2]:
def predict_old(model,train_series_dict,valid_series_dict,covariates,train_series_covars):
    if covariates == True:
        pred = model.predict(n=1, series=list(train_series_dict.values()), past_covariates=list(train_series_covars.values()))
    else:
        pred = model.predict(n=1, series=list(train_series_dict.values()),num_samples=1)
    last_values = {
        key: ts.values()[-1].flatten()[0]
        for key, ts in valid_series_dict.items()
    }
    # Extract the predicted values from pred (assuming the order matches valid_series_dict keys)
    pred_values = [ts.values().flatten()[0] for ts in pred]
    # Initialize lists to store actual and predicted values
    actual_values = []
    predicted_values = []
    keys = []

    # Extract actual and predicted values
    for (key, last_value), pred_value in zip(last_values.items(), pred_values):
        actual_values.append(last_value)
        predicted_values.append(pred_value)
        keys.append(key)

    # Convert lists to numpy arrays for easier computation
    actual_values_np = np.array(actual_values)
    predicted_values_np = np.array(predicted_values)

    # Calculate MAE
    mae = np.mean(np.abs(actual_values_np - predicted_values_np))
    mse = np.mean((actual_values_np - predicted_values_np) ** 2)
    # Calculate RMSE
    rmse = np.sqrt(np.mean((actual_values_np - predicted_values_np) ** 2))

    # Calculate MAPE
    mape = np.mean(np.abs((actual_values_np - predicted_values_np) / actual_values_np)) * 100
    denominator = np.abs(actual_values_np) + np.abs(predicted_values_np)
    denominator[denominator == 0] = 1
    smape = np.mean(np.abs(actual_values_np - predicted_values_np) / denominator) * 100

    # Print the metrics
    print(f"MAE: {mae}")
    print(f"MASE: {mse}")
    print(f"RMSE: {rmse}")
    #print(f"MAPE: {mape}%")

    print(f"SMAPE: {smape}%")
    return predicted_values, actual_values, keys



In [ ]:

def predict(model, train_series_dict, valid_series_dict, covariates, train_series_covars):
    if covariates == True:
        pred = model.predict(n=1, series=list(train_series_dict.values()), past_covariates=list(train_series_covars.values()))
    else:
        pred = model.predict(n=1, series=list(train_series_dict.values()), num_samples=5)

    # Extract the last actual values from the validation series
    last_values = {
        key: ts.values()[-1].flatten()[0]
        for key, ts in valid_series_dict.items()
    }

    # Extract the predicted values from pred (assuming the order matches valid_series_dict keys)
    # Calculate the median of the predictions (axis 0 is the time dimension, axis 1 is the sample dimension)
    pred_medians = [np.median(ts.values(), axis=1)[0] for ts in pred]

    # Initialize lists to store actual and predicted values
    actual_values = []
    predicted_values = []
    keys = []

    # Extract actual and predicted values
    for (key, last_value), pred_median in zip(last_values.items(), pred_medians):
        actual_values.append(last_value)
        predicted_values.append(pred_median)
        keys.append(key)

    # Convert lists to numpy arrays for easier computation
    actual_values_np = np.array(actual_values)
    predicted_values_np = np.array(predicted_values)

    # Calculate MAE
    mae = np.mean(np.abs(actual_values_np - predicted_values_np))
    mse = np.mean((actual_values_np - predicted_values_np) ** 2)
    # Calculate RMSE
    rmse = np.sqrt(np.mean((actual_values_np - predicted_values_np) ** 2))

    # Calculate MAPE
    mape = np.mean(np.abs((actual_values_np - predicted_values_np) / actual_values_np)) * 100
    
    # Calculate SMAPE
    denominator = np.abs(actual_values_np) + np.abs(predicted_values_np)
    denominator[denominator == 0] = 1  # To avoid division by zero
    smape = np.mean(np.abs(actual_values_np - predicted_values_np) / denominator) * 100

    # Print the metrics
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAPE: {mape}%")
    print(f"SMAPE: {smape}%")

    return predicted_values, actual_values, keys, pred


In [1]:
def data_processing(data):
    start_prediction,restrict_station,station,value_to_predict,resample_rate,prediction_length,max_train_length,min_train_length,min_ts_length,metric,imputation,persistence_use_case,persistence_time_lag,min_value,padding,covariates,covariates_list,model_library = config_load()
    filtered_data, filtered_data_full = process_ts(data,start_prediction,value_to_predict,min_value,persistence_use_case,persistence_time_lag)
    if restrict_station == True:
        df = filter_station(filtered_data,station)
        resampled_df = resample_ts(df,resample_rate,min_ts_length,imputation,max_train_length,value_to_predict,covariates)
    else:
        resampled_df = resample_ts(filtered_data,resample_rate,min_ts_length,imputation,max_train_length,value_to_predict,covariates)

    unique_encounter_ids = resampled_df['encounter_id'].unique()
    train_encounter_ids, test_encounter_ids = train_test_split(unique_encounter_ids, test_size=0.2, random_state=42)
    train_df = resampled_df[resampled_df['encounter_id'].isin(train_encounter_ids)]
    test_df = resampled_df[resampled_df['encounter_id'].isin(test_encounter_ids)]

    valid_series_dict, test_series_dict, train_series_dict = ts_dictionary(train_df,prediction_length)
    valid_series_covars, test_series_covars, train_series_covars = ts_dictionary_covariates(train_df, prediction_length,covariates_list)

    valid_series_dict_test, test_series_dict_test, train_series_dict_test = ts_dictionary(test_df,prediction_length)
    valid_series_covars_test, test_series_covars_test, train_series_covars_test = ts_dictionary_covariates(test_df, prediction_length,covariates_list)

    valid_series_dict_full, test_series_dict_full, train_series_dict_full = ts_dictionary(resampled_df,prediction_length)
    valid_series_covars_full, test_series_covars_full, train_series_covars_full = ts_dictionary_covariates(resampled_df, prediction_length,covariates_list)

    model_nbeats, model_xgb, model_lgb, model_tide,model_aarima = model_configs(min_train_length,prediction_length, max_train_length,padding)

    # Pad the series
    if padding == True:
        train_series_dict = pad_series_dict(train_series_dict, max_train_length, pad_value=0)
        train_series_covars = pad_series_dict(train_series_covars, max_train_length, pad_value=0)
        train_series_dict_test = pad_series_dict(train_series_dict_test, max_train_length, pad_value=0)
        train_series_covars_test = pad_series_dict(train_series_covars_test, max_train_length, pad_value=0)
        train_series_dict_full = pad_series_dict(train_series_dict_full, max_train_length, pad_value=0)
        train_series_covars_full = pad_series_dict(train_series_covars_full, max_train_length, pad_value=0)   

    return train_series_dict_full, train_series_covars_full,valid_series_dict_full,train_series_covars,resampled_df



In [ ]:
def data_processing_training(data,start_prediction,value_to_predict,min_value,persistence_use_case,persistence_time_lag,station,resample_rate,min_ts_length,imputation,max_train_length,prediction_length,covariates_list,min_train_length,padding,restrict_station, covariates):


    filtered_data, filtered_data_full = process_ts(data,start_prediction,value_to_predict,min_value,persistence_use_case,persistence_time_lag)
    if restrict_station == True:
        df = filter_station(filtered_data,station)
        resampled_df = resample_ts(df,resample_rate,min_ts_length,imputation,max_train_length,value_to_predict,covariates)
    else:
        resampled_df = resample_ts(filtered_data,resample_rate,min_ts_length,imputation,max_train_length,value_to_predict,covariates)

    unique_encounter_ids = resampled_df['encounter_id'].unique()
    train_encounter_ids, test_encounter_ids = train_test_split(unique_encounter_ids, test_size=0.2, random_state=42)
    train_df = resampled_df[resampled_df['encounter_id'].isin(train_encounter_ids)]
    test_df = resampled_df[resampled_df['encounter_id'].isin(test_encounter_ids)]

    valid_series_dict, test_series_dict, train_series_dict = ts_dictionary(train_df,prediction_length)
    valid_series_covars, test_series_covars, train_series_covars = ts_dictionary_covariates(train_df, prediction_length,covariates_list)

    valid_series_dict_test, test_series_dict_test, train_series_dict_test = ts_dictionary(test_df,prediction_length)
    valid_series_covars_test, test_series_covars_test, train_series_covars_test = ts_dictionary_covariates(test_df, prediction_length,covariates_list)

    valid_series_dict_full, test_series_dict_full, train_series_dict_full = ts_dictionary(resampled_df,prediction_length)
    valid_series_covars_full, test_series_covars_full, train_series_covars_full = ts_dictionary_covariates(resampled_df, prediction_length,covariates_list)

    model_nbeats, model_xgb, model_lgb, model_tide,model_aarima = model_configs(min_train_length,prediction_length, max_train_length,padding)

    # Pad the series
    if padding == True:
        train_series_dict = pad_series_dict(train_series_dict, max_train_length, pad_value=0)
        train_series_covars = pad_series_dict(train_series_covars, max_train_length, pad_value=0)
        train_series_dict_test = pad_series_dict(train_series_dict_test, max_train_length, pad_value=0)
        train_series_covars_test = pad_series_dict(train_series_covars_test, max_train_length, pad_value=0)
        train_series_dict_full = pad_series_dict(train_series_dict_full, max_train_length, pad_value=0)
        train_series_covars_full = pad_series_dict(train_series_covars_full, max_train_length, pad_value=0)   

    return train_series_dict_full, train_series_covars_full,valid_series_dict_full, train_series_dict,train_series_covars,train_series_dict_test,train_series_covars_test,valid_series_dict,valid_series_dict_test,valid_series_covars,valid_series_covars_test,model_nbeats, model_xgb, model_lgb, model_tide,model_aarima,resampled_df



In [ ]:
# Load data from csv files
def load_data_from_csv_and_merge_no_covars(df_crp,df_ab_groups):
    try:
        # Dropping NaN values from specific dataframes
        #df_bili = df_bili.dropna()
        df_crp = df_crp.dropna()
        #df_ggt = df_ggt.dropna()
        #df_hb = df_hb.dropna()
        #df_krea = df_krea.dropna()
        #df_leua = df_leua.dropna()
        #df_temp = df_temp.dropna()
        # List of dataframes
        dataframes = [df_crp]

        def process_timestamps(df, time_column='recorded_time'):
            try:
                df[time_column] = pd.to_datetime(df[time_column], utc=True)
                df['recorded_date'] = df[time_column].dt.date
            except Exception as e:
                print(f"Error processing timestamps for dataframe: {e}")

        # Process timestamps for each dataframe
        for df in dataframes:
            process_timestamps(df)

        try:
            df_ab_groups['recorded_time'] = pd.to_datetime(df_ab_groups['recorded_time'], utc=True)
            df_ab_grouped = df_ab_groups.groupby('encounter_id')['recorded_time'].agg(['min', 'max']).reset_index()
            df_ab_grouped.columns = ['encounter_id', 'min_timestamp', 'max_timestamp']
            df_crp = pd.merge(df_crp, df_ab_grouped, on='encounter_id', how='left')
            df_crp['intervention'] = ((df_crp['recorded_time'] >= df_crp['min_timestamp']) &
                                       (df_crp['recorded_time'] <= df_crp['max_timestamp'])).astype(int)
            df_crp = df_crp.drop(columns=['min_timestamp', 'max_timestamp'])
        except Exception as e:
            print(f"Error processing df_ab_groups: {e}")

        # Rename values
        try:
            df_crp = df_crp.rename(columns={'value': 'crp'})
        except Exception as e:
            print(f"Error renaming columns: {e}")

        return df_crp, df_ab_groups
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [ ]:
# Create time lags
def create_merge_no_covars(df_crp, df_ab_groups):
    try:
        # Convert 'recorded_time' column to datetime if it's not already
        df_crp['recorded_time'] = pd.to_datetime(df_crp['recorded_time'], utc=True)
        
        # Group by 'encounter_id'
        grouped = df_crp.groupby('encounter_id')

        # Define functions to create time lag indicators
        def within_24_hours(group):
            try:
                intervention_1_time = group.loc[group['intervention'] == 1, 'recorded_time'].min()
                return (group['recorded_time'] >= intervention_1_time) & ((group['recorded_time'] - intervention_1_time).dt.total_seconds() <= 24*60*60)
            except Exception as e:
                print(f"Error in within_24_hours: {e}")
                return pd.Series([False]*len(group), index=group.index)

        def within_24_to_48_hours(group):
            try:
                intervention_1_time = group.loc[group['intervention'] == 1, 'recorded_time'].min()
                return ((group['recorded_time'] - intervention_1_time).dt.total_seconds() > 24*60*60) & ((group['recorded_time'] - intervention_1_time).dt.total_seconds() <= 48*60*60)
            except Exception as e:
                print(f"Error in within_24_to_48_hours: {e}")
                return pd.Series([False]*len(group), index=group.index)

        def within_48_to_72_hours(group):
            try:
                intervention_1_time = group.loc[group['intervention'] == 1, 'recorded_time'].min()
                return ((group['recorded_time'] - intervention_1_time).dt.total_seconds() > 48*60*60) & ((group['recorded_time'] - intervention_1_time).dt.total_seconds() <= 72*60*60)
            except Exception as e:
                print(f"Error in within_48_to_72_hours: {e}")
                return pd.Series([False]*len(group), index=group.index)

        def prior_to_24_hours(group):
            try:
                intervention_1_time = group.loc[group['intervention'] == 1, 'recorded_time'].min()
                prior_intervention_time = intervention_1_time - pd.Timedelta(hours=24)
                return (group['recorded_time'] >= prior_intervention_time) & (group['recorded_time'] < intervention_1_time)
            except Exception as e:
                print(f"Error in prior_to_24_hours: {e}")
                return pd.Series([False]*len(group), index=group.index)

        # Apply the functions to create new columns
        try:
            df_crp['time_lag_2'] = grouped.apply(within_24_hours).reset_index(level=0, drop=True)
            df_crp['time_lag_1'] = grouped.apply(within_24_to_48_hours).reset_index(level=0, drop=True)
            df_crp['time_lag_target'] = grouped.apply(within_48_to_72_hours).reset_index(level=0, drop=True)
            time_lag_prior_24_hours_series = grouped.apply(prior_to_24_hours).reset_index(level=0, drop=True)
        except Exception as e:
            print(f"Error creating time lag columns: {e}")


        # Base dataframe
        base_df = df_crp[['encounter_id', 'subject_reference', 'recorded_time', 'crp', 'intervention', 'time_lag_1', 'time_lag_2', 'time_lag_target']].sort_values('recorded_time')
        #base_df = df_crp[['encounter_id', 'subject_reference', 'recorded_time', 'crp', 'intervention', 'time_lag_1', 'time_lag_2', 'time_lag_target']].sort_values('recorded_time')

        data = base_df
        
        return data
    except Exception as e:
        print(f"An error occurred in create_time_lags: {e}")
        return None


In [ ]:
import random
import time

def plot_random_ts(random_seed,predicted_values,actual_values,keys,valid_series_dict):
    all_values = list(zip(keys, predicted_values, actual_values))
    if random_seed != None:
        random.seed(random_seed)
    else:
        seed = int(time.time())  # Current time in seconds since the epoch
        random.seed(seed)
    all_identifiers = [entry[0] for entry in all_values]
    random_identifier = random.choice(all_identifiers)
    # Function to find the entry by identifier
    def find_entry_by_identifier(values_list, search_id):
        for entry in values_list:
            if entry[0] == search_id:
                return entry
        return None  # Return None if the identifier is not found
    
    # Find the entry
    result = find_entry_by_identifier(all_values, random_identifier)
    #print(result)
    specific_entry = valid_series_dict[result[0]]
    df = specific_entry.pd_dataframe()
    df = df.reset_index()
    # Extract the additional point
    additional_time = df['recorded_time'].iloc[-1]  # Last value in the recorded_time column
    additional_crp = result[1]
    plot_title = result[0]
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df['recorded_time'], df['crp'], marker='o', linestyle='-', color='black', label='CRP Data')
    plt.scatter(additional_time, additional_crp, color='red', edgecolor='black', s=100, label='Additional Point')  # Adding the extra point
    plt.xlabel('Recorded Time')
    plt.ylabel('CRP')
    plt.title(plot_title)
    plt.grid(True)

    plt.xticks(rotation=45)
    plt.legend()  # Add a legend to distinguish between data series
    plt.tight_layout()  # Adjusts the plot to fit everything nicely

    # Show the plot
    plt.show()



In [ ]:
import random
import time
import matplotlib.pyplot as plt

def plot_random_ts_line(random_seed, predicted_values, actual_values, keys, valid_series_dict):
    all_values = list(zip(keys, predicted_values, actual_values))
    if random_seed is not None:
        random.seed(random_seed)
    else:
        seed = int(time.time())  # Current time in seconds since the epoch
        random.seed(seed)
    all_identifiers = [entry[0] for entry in all_values]
    random_identifier = random.choice(all_identifiers)
    
    # Function to find the entry by identifier
    def find_entry_by_identifier(values_list, search_id):
        for entry in values_list:
            if entry[0] == search_id:
                return entry
        return None  # Return None if the identifier is not found
    
    # Find the entry
    result = find_entry_by_identifier(all_values, random_identifier)
    
    specific_entry = valid_series_dict[result[0]]
    df = specific_entry.pd_dataframe()
    df = df.reset_index()
    
    # Extract the additional point
    additional_time = df['recorded_time'].iloc[-1]  # Last value in the recorded_time column
    additional_crp = result[1]
    plot_title = result[0]
    
    # Penultimate point
    penultimate_time = df['recorded_time'].iloc[-2]
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df['recorded_time'], df['crp'], marker='o', linestyle='-', color='black', label='CRP Data')
    plt.scatter(additional_time, additional_crp, color='red', edgecolor='black', s=100, label='Additional Point')  # Adding the extra point
    
    # Add the vertical dotted blue line at the penultimate point
    plt.axvline(x=penultimate_time, color='blue', linestyle='--', label='Start of antibiotic therapy')
    
    plt.xlabel('Recorded Time')
    plt.ylabel('CRP')
    plt.title(plot_title)
    plt.grid(True)
    
    plt.xticks(rotation=45)
    plt.legend()  # Add a legend to distinguish between data series
    plt.tight_layout()  # Adjusts the plot to fit everything nicely
    
    # Show the plot
    plt.show()


In [ ]:
import random
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error

def plot_random_ts_proba(random_seed, predicted_values, actual_values, keys, valid_series_dict):
    all_values = list(zip(keys, predicted_values, actual_values))
    if random_seed is not None:
        random.seed(random_seed)
    else:
        seed = int(time.time())  # Current time in seconds since the epoch
        random.seed(seed)
    all_identifiers = [entry[0] for entry in all_values]
    random_identifier = random.choice(all_identifiers)
    
    # Function to find the entry by identifier
    def find_entry_by_identifier(values_list, search_id):
        for entry in values_list:
            if entry[0] == search_id:
                return entry
        return None  # Return None if the identifier is not found
    
    # Find the entry
    result = find_entry_by_identifier(all_values, random_identifier)
    
    specific_entry = valid_series_dict[result[0]]
    df = specific_entry.pd_dataframe()
    df = df.reset_index()
    
    # Extract the additional point
    additional_time = df['recorded_time'].iloc[-1]  # Last value in the recorded_time column
    additional_crp = result[1]['q0.5']
    plot_title = result[0]
    
    # Penultimate point
    penultimate_time = df['recorded_time'].iloc[-2]
    
    # Extract actual and predicted values for the specific series
    actual_series_values = df['crp'].values
    
    # Extract predicted quantiles
    predicted_series_q05 = result[1]['q0.05']
    predicted_series_q50 = result[1]['q0.5']
    predicted_series_q95 = result[1]['q0.95']

    # Ensure predicted_series_values are array-like structures
    predicted_series_q05 = np.array(predicted_series_q05)
    predicted_series_q50 = np.array(predicted_series_q50)
    predicted_series_q95 = np.array(predicted_series_q95)

    # Calculate MAE for the 0.5 quantile
    mae = mean_absolute_error(actual_series_values, predicted_series_q50)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df['recorded_time'], df['crp'], marker='o', linestyle='-', color='black', label='CRP Data')
    plt.scatter(df['recorded_time'].iloc[-1], additional_crp, color='red', edgecolor='black', s=100, label='Additional Point')  # Adding the extra point
    
    # Add the vertical dotted blue line at the penultimate point
    plt.axvline(x=penultimate_time, color='blue', linestyle='--', label='Start of antibiotic therapy')
    
    # Plot the 0.5 quantile as points
    plt.scatter(df['recorded_time'], predicted_series_q50, color='green', label='Predicted q0.5')
    
    # Plot the 0.05 and 0.95 quantiles as a shaded area
    plt.fill_between(df['recorded_time'], predicted_series_q05, predicted_series_q95, color='orange', alpha=0.3, label='0.05-0.95 interval')
    
    plt.xlabel('Recorded Time')
    plt.ylabel('CRP')
    plt.title(f"{plot_title} (MAE: {mae:.2f})")  # Display MAE in the title
    plt.grid(True)
    
    plt.xticks(rotation=45)
    plt.legend()  # Add a legend to distinguish between data series
    plt.tight_layout()  # Adjusts the plot to fit everything nicely
    
    # Show the plot
    plt.show()

# Example usage:
# plot_random_ts(random_seed, predicted_values, actual_values, keys, valid_series_dict)


In [ ]:
import matplotlib.pyplot as plt

def plot_ts_key(selected_key, predicted_values, actual_values, keys, valid_series_dict):
    all_values = list(zip(keys, predicted_values, actual_values))
    
    # Function to find the entry by identifier
    def find_entry_by_identifier(values_list, search_id):
        for entry in values_list:
            if entry[0] == search_id:
                return entry
        return None  # Return None if the identifier is not found
    
    # Find the entry
    result = find_entry_by_identifier(all_values, selected_key)
    if result is None:
        raise ValueError(f"Key '{selected_key}' not found in the provided data.")
    
    specific_entry = valid_series_dict[result[0]]
    df = specific_entry.pd_dataframe()
    df = df.reset_index()
    
    # Extract the additional point
    additional_time = df['recorded_time'].iloc[-1]  # Last value in the recorded_time column
    additional_crp = result[1]
    plot_title = result[0]
    
    # Penultimate point
    penultimate_time = df['recorded_time'].iloc[-2]
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df['recorded_time'], df['crp'], marker='o', linestyle='-', color='black', label='CRP Data')
    plt.scatter(additional_time, additional_crp, color='red', edgecolor='black', s=100, label='Additional Point')  # Adding the extra point
    
    # Add the vertical dotted blue line at the penultimate point
    plt.axvline(x=penultimate_time, color='blue', linestyle='--', label='Start of antibiotic therapy')
    
    plt.xlabel('Recorded Time')
    plt.ylabel('CRP')
    plt.title(plot_title)
    plt.grid(True)
    
    plt.xticks(rotation=45)
    plt.legend()  # Add a legend to distinguish between data series
    plt.tight_layout()  # Adjusts the plot to fit everything nicely
    
    # Show the plot
    plt.show()


In [ ]:
import numpy as np

def predict_proba(model, train_series_dict, valid_series_dict, covariates, train_series_covars):
    if covariates:
        pred = model.predict(n=1, series=list(train_series_dict.values()), past_covariates=list(train_series_covars.values()))
    else:
        pred = model.predict(n=1, series=list(train_series_dict.values()), num_samples=500)

    pred_quantile = ([ts.quantile(0.5) for ts in pred])
    # Extract the last actual values from the validation series
    last_values = {
        key: ts.values()[-1].flatten()[0]
        for key, ts in valid_series_dict.items()
    }

    # Convert the list of TimeSeries to a NumPy array
    pred_arrays = np.array([ts.values() for ts in pred_quantile])  # Shape should be (num_series, 1, num_samples)
    #pred_arrays = ([ts.values() for ts in pred])
    # Debugging output
    #print(f"Shape of pred_arrays: {pred_arrays.shape}")  # Debugging line

    # Calculate the median prediction for each series
    if pred_arrays.ndim == 3:
    #    # Take the median across the sample dimension (axis 2)
        pred_medians = np.median(pred_arrays, axis=2)[:, 0]  # Shape (num_series,)
    else:
    #    # Handle case where num_samples is 1
        pred_medians = np.squeeze(pred_arrays)  # Shape (num_series,)

    # Initialize lists to store actual and predicted values
    actual_values = []
    predicted_values = []
    keys = []

    # Extract actual and predicted values
    for (key, last_value), pred_arrays in zip(last_values.items(), pred_medians):
        actual_values.append(last_value)
        predicted_values.append(pred_medians)
        keys.append(key)

    # Convert lists to numpy arrays for easier computation
    actual_values_np = np.array(actual_values)
    predicted_values_np = np.array(predicted_values)

    # Calculate MAE
    mae = np.mean(np.abs(actual_values_np - predicted_values_np))
    mse = np.mean((actual_values_np - predicted_values_np) ** 2)
    # Calculate RMSE
    rmse = np.sqrt(np.mean((actual_values_np - predicted_values_np) ** 2))

    # Calculate MAPE
    mape = np.mean(np.abs((actual_values_np - predicted_values_np) / actual_values_np)) * 100
    
    # Calculate SMAPE
    denominator = np.abs(actual_values_np) + np.abs(predicted_values_np)
    denominator[denominator == 0] = 1  # To avoid division by zero
    smape = np.mean(np.abs(actual_values_np - predicted_values_np) / denominator) * 100

    # Print the metrics
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAPE: {mape}%")
    print(f"SMAPE: {smape}%")
    pred_ci_down = ([ts.quantile(0.05) for ts in pred])
    pred_ci_up = ([ts.quantile(0.95) for ts in pred])
    pred_ci_down = np.array([ts.values() for ts in pred_ci_down])
    pred_ci_up = np.array([ts.values() for ts in pred_ci_up])
    return predicted_values, actual_values, keys, pred_quantile,pred_ci_down,pred_ci_up


In [ ]:


def plot_random_ts_proba(key_index, predicted_values, actual_values, keys, valid_series_dict_full, pred_ci_up, pred_ci_down):
    if key_index == None:
        #key_index = random.choice(keys)
        key_index = random.randint(0, len(keys) - 1)
        print(key_index)
    predicted_values = [value for array in predicted_values for value in array]
   # Extract the entry corresponding to the specified key index
    key = keys[key_index]
    predicted_value = predicted_values[key_index]
    actual_value = actual_values[key_index]

    specific_entry = valid_series_dict_full[key]
    df = specific_entry.pd_dataframe()
    df = df.reset_index()

    # Extract the additional point
    additional_time = df['recorded_time'].iloc[-1]  # Last value in the recorded_time column
    additional_crp = predicted_value
    plot_title = "Recorded and predicted CRP values for ID "+key

    # Extract the index of the additional point for CI values
    additional_index = df.index[df['recorded_time'] == additional_time].tolist()[0]

    additional_ci_up = pred_ci_up[key_index][0][0]
    additional_ci_down2 = pred_ci_down[key_index][0][0]
    # Penultimate point
    penultimate_time = df['recorded_time'].iloc[-2]
    mae = np.mean(np.abs(actual_value - predicted_value))
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df['recorded_time'], df['crp'], marker='o', linestyle='-', color='black', label='Recorded CRP values')
    plt.scatter(additional_time, additional_crp, color='red', edgecolor='black', s=100, label='Predicted CRP value')  # Adding the extra point
    plt.errorbar(additional_time, additional_crp, yerr=[[additional_crp-additional_ci_down2], [additional_ci_up-additional_crp]], fmt='o', color='red', capsize=5, label='95% CI')

    # Add the vertical dotted blue line at the penultimate point
    plt.axvline(x=penultimate_time, color='blue', linestyle='--', label='Start of antibiotic therapy')

    plt.xlabel('Recorded Time')
    plt.ylabel('CRP')
    #plt.title(plot_title)
    plt.title(f"{key} (MAE: {mae:.2f})")  # Display MAE in the title
    plt.grid(True)

    plt.xticks(rotation=45)
    plt.legend()  # Add a legend to distinguish between data series
    plt.tight_layout()  # Adjusts the plot to fit everything nicely

    # Show the plot
    plt.show() 
    

In [ ]:
def config_load():
    # Load configuration from YAML-file
    # No changes to configuration.yaml are required for executing the pretrained models.
    with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)

    # Access configuration values
    start_prediction = config['model_configurations']['start_prediction']
    restrict_station = config['model_configurations']['restrict_station']
    station = config['model_configurations']['station']
    value_to_predict = config['model_configurations']['value_to_predict']
    resample_rate = config['model_configurations']['resample_rate']
    prediction_length = config['model_configurations']['prediction_length']
    max_train_length = config['model_configurations']['max_train_length']
    min_train_length = config['model_configurations']['min_train_length']
    min_ts_length = config['model_configurations']['min_ts_length']
    metric = config['model_configurations']['metric']
    imputation = config['model_configurations']['imputation']
    persistence_use_case = config['model_configurations']['persistence_use_case']
    persistence_time_lag = config['model_configurations']['persistence_time_lag']
    min_value = config['model_configurations']['min_value']
    padding = config['model_configurations']['padding']
    covariates = config['model_configurations']['covariates']
    covariates_list = config['model_configurations']['covariates_list']
    model_library = config['model_configurations']['model_library']
    return start_prediction,restrict_station,station,value_to_predict,resample_rate,prediction_length,max_train_length,min_train_length,min_ts_length,metric,imputation,persistence_use_case,persistence_time_lag,min_value,padding,covariates,covariates_list,model_library

In [2]:
def autogluon_data(resampled_df,predictor):
    with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)
    prediction_length = config['model_configurations']['prediction_length']
    value_to_predict = config['model_configurations']['value_to_predict']

    # Split the data into train and test sets
    def split_train_test(group):
        test_rows = group.nlargest(prediction_length, 'recorded_time')
        train_rows = group.drop(test_rows.index)
        return train_rows, test_rows
    
    train_list = []
    test_list = []
    
    # Group by encounter_id and split each group
    for name, group in resampled_df.groupby('encounter_id'):
        train_rows, test_rows = split_train_test(group)
        train_list.append(train_rows)
        test_list.append(test_rows)
    
    # Concatenate the lists into DataFrames
    train_data = pd.concat(train_list).reset_index(drop=True)
    test_data = pd.concat(test_list).reset_index(drop=True)
    train_data_ = train_data[['encounter_id', 'recorded_time', value_to_predict]]
    train_data_['recorded_time'] = train_data_['recorded_time'].dt.tz_localize(None)
    test_data_ = resampled_df[['encounter_id', 'recorded_time', value_to_predict]]
    test_data_['recorded_time'] = test_data_['recorded_time'].dt.tz_localize(None)
    train_data = TimeSeriesDataFrame.from_data_frame(
        train_data_,
        id_column="encounter_id",
        timestamp_column="recorded_time"
    )
    test_data = TimeSeriesDataFrame.from_data_frame(
        test_data_,
        id_column="encounter_id",
        timestamp_column="recorded_time"
    )
    predictions = predictor.predict(train_data,model='WeightedEnsemble')
    predictions_ = predictions[['mean','0.05','0.95']]
    merged_data = test_data.join(predictions_, how="inner")
    # Extract actual and predicted values
    actuals = merged_data['crp'].values
    predictions = merged_data['mean'].values

    # MAE: Mean Absolute Error
    mae = mean_absolute_error(actuals, predictions)

    # MSE: Mean Squared Error
    mse = mean_squared_error(actuals, predictions)

    # RMSE: Root Mean Squared Error
    rmse = np.sqrt(mse)

    # MAPE: Mean Absolute Percentage Error
    mape = mean_absolute_percentage_error(actuals, predictions) * 100

    # SMAPE: Symmetric Mean Absolute Percentage Error
    smape = np.mean(2 * np.abs(actuals - predictions) / (np.abs(actuals) + np.abs(predictions))) * 100

    # Display the results
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.4f}%")
    print(f"SMAPE: {smape:.4f}%")

    return predictions,train_data, test_data, merged_data

In [ ]:

def plot_random_ts_proba_gluon(random_seed, test_data, merged_data):
    # Get the unique identifiers (first level of MultiIndex)
    all_identifiers = merged_data.index.get_level_values(0).unique()
    
    # Set random seed
    if random_seed is not None:
        random.seed(random_seed)
    else:
        seed = int(time.time())
        random.seed(seed)

    # Choose a random identifier from merged_data (where predictions exist)
    random_identifier = random.choice(all_identifiers)

    # Filter the historical data for the selected identifier from test_data
    historic_data = test_data.loc[random_identifier]

    # Filter the prediction data for the selected identifier from merged_data
    prediction_data = merged_data.loc[random_identifier]

    # Extract the time series data for the chosen identifier
    recorded_time = historic_data.index
    actual_series_values = historic_data['crp'].values

    # Extract prediction and confidence intervals
    prediction_time = prediction_data.index
    predicted_series_q05 = prediction_data['0.05'].values
    predicted_series_q05 = np.maximum(predicted_series_q05, 0)
    predicted_series_q50 = prediction_data['mean'].values
    predicted_series_q95 = prediction_data['0.95'].values

    # Calculate MAE for the 0.5 quantile
    mae = mean_absolute_error(actual_series_values[-len(predicted_series_q50):], predicted_series_q50)

    # Identify the penultimate point in the historical data
    penultimate_time = recorded_time[-2]

    # Extract the additional point (last predicted point)
    additional_time = prediction_time[-1]
    additional_crp = predicted_series_q50[-1]
    additional_ci_down = predicted_series_q05[-1]
    additional_ci_up = predicted_series_q95[-1]
    
    # Plotting
    plt.figure(figsize=(10, 6))

    # Plot the actual historical CRP data
    plt.plot(recorded_time, actual_series_values, marker='o', linestyle='-', color='black', label='CRP Data')

    # Add the vertical dotted blue line at the penultimate point
    plt.axvline(x=penultimate_time, color='blue', linestyle='--', label='Start of antibiotic therapy')

    # Plot the predicted q0.5 (median) as points
    plt.scatter(prediction_time, predicted_series_q50, color='red', edgecolor='black', s=100, label='Predicted CRP value')

    # Add the error bar for the last predicted point with 95% CI
    plt.errorbar(additional_time, additional_crp, 
                 yerr=[[additional_crp - additional_ci_down], [additional_ci_up - additional_crp]], 
                 fmt='o', color='red', capsize=5, label='95% CI')
    
    plt.xlabel('Recorded Time')
    plt.ylabel('CRP')
    plt.title(f"{random_identifier} (MAE: {mae:.2f})")  # Display MAE in the title
    plt.grid(True)
    
    plt.xticks(rotation=45)
    plt.legend()  # Add a legend to distinguish between data series
    plt.tight_layout()  # Adjusts the plot to fit everything nicely
    
    # Show the plot
    plt.show()